In [1]:
import os
import pandas as pd

In [2]:
# put IDs in set
rootdir = 'C:/Users/tvdbo/Documents/Projecten/AAquant'
identifier_set = set()
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith("protein.tsv"):
            with open(os.path.join(subdir, file), "r") as in_f:
                next(in_f)
                for line in in_f:
                    _, _, _, identifier, _ = line.split("\t", maxsplit=4)
                    identifier_set.add(identifier)

In [3]:
# write IDs to file
with open("identifiers.txt", "w") as out_f:
    for identifier in identifier_set:
        print(identifier, file=out_f)

In [ ]:
# TO DO MANUALLY, NOT IN SCRIPT
# Put all identifiers in the UniProt batch retrieve tool (https://www.uniprot.org/id-mapping). 
# UniProtKB AC/ID -> UniProtKB
# Enable GO columns 
# Save as "ProteinID_GO.tsv" 
# Afterwards, it turned out sequences where necessary as well
# Repeat process (enable sequences), save as "ProteinID_sequence.tsv"

In [3]:
# load GO terms into separate dict
GOid_dict = dict()
GOcell_dict = dict()
GObiol_dict = dict()
GOmol_dict = dict()
with open("ProteinID_GO.tsv", "r") as in_f:
    next(in_f)
    for line in in_f:
        identifier, _, _, _, _, _, GOid, GOcell, GObiol, GOmol, _ = line.split("\t", maxsplit=10)
        GOid_dict[identifier] = GOid
        GOcell_dict[identifier] = GOcell
        GObiol_dict[identifier] = GObiol
        GOmol_dict[identifier] = GOmol

# load protein sequences into dict
seq_dict = dict()
with open("ProteinID_sequence.tsv", "r") as in_f:
    next(in_f)
    for line in in_f:
        identifier, _, _, sequence = line.split("\t", maxsplit=3)
        seq_dict[identifier] = sequence

In [4]:
# add GO terms to new file protein_GO_seq.tsv
rootdir = 'C:/Users/tvdbo/Documents/Projecten/AAquant'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith("protein.tsv"):
            f_check = True
            #print(os.path.join(subdir, file))
            df = pd.read_csv(os.path.join(subdir, file), sep='\t', header=0)
            df['GO_ids']= df['Protein ID'].map(GOid_dict)
            df['GO_cellular_component']= df['Protein ID'].map(GOcell_dict)
            df['GO_biological_process']= df['Protein ID'].map(GObiol_dict)
            df['GO_molecular_function']= df['Protein ID'].map(GOmol_dict)
            df['Sequence']= df['Protein ID'].map(seq_dict)
            df.to_csv(os.path.join(subdir, file.replace(".tsv", "_GO_seq.tsv")), sep='\t', index=False)
